## Continuation Passing Style and Scala Futures

In Scala, Futures are used to wrap asynchronous computation. When you need a result, you can wait for the Future to have a result using `Await.result(future)`. The almond scala kernel updates the Future automatically. Futures work in a similar way to CPS because when the result is available, your code is called with that result as an argument.

Here we create a function that takes a continuation `k` and uses a library to send a GET request to a url. We wrap the call in a Future and map results to their response text and then call `k` on each result.

In [0]:
import $ivy.`com.lihaoyi::requests:0.1.4`
import scala.concurrent.{ Future, Await }
import scala.concurrent.ExecutionContext.Implicits.global

def http_get(url: String, k: String => String): Future[String] =
    Future(requests.get(url)).map(_.text).map(k)

Downloaded https://repo1.maven.org/maven2/com/lihaoyi/requests_2.13/0.1.4/requests_2.13-0.1.4.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/requests_2.13/0.1.4/requests_2.13-0.1.4.pom.sha1
Failed to resolve ivy dependencies:Error downloading com.lihaoyi:requests_2.13:0.1.4
  not found: /Users/kschuygon/.ivy2/local/com.lihaoyi/requests_2.13/0.1.4/ivys/ivy.xml
  not found: https://repo1.maven.org/maven2/com/lihaoyi/requests_2.13/0.1.4/requests_2.13-0.1.4.pom

: 

In [0]:
val shakira_url = "http://www.mocky.io/v2/5c6fa3fc3400004e5f8931a6"

val res = http_get(shakira_url, _ + " lie")

cmd0.sc:3: not found: value http_get
val res = http_get(shakira_url, _ + " lie")
          ^Compilation Failed

: 

## Parametric Polymorphism and Generic Types

Parametric Polymorphism (https://en.wikipedia.org/wiki/Parametric_polymorphism) is a language feature that allows us to write functions and classes with arguments and member variables that have types that are not yet defined. This means that such functions can be used on many different types (they are "generic" or "polymorphic" functions or classes). The "parametric" part means that we need to parameterize the unspecified types using symbols meaning "these are the same type."

In [1]:
// id takes an argument of type T and returns that argument
def id[T](x: T): T = x


defined function id

In [2]:
val example = id(List(2))

example: List[Int] = List(2)

In [3]:
case class MyClass[T](t: T) {
    def get: T = t
}

defined class MyClass

In [4]:
val x: Int = MyClass(2).get
val y: List[Int] = MyClass(List(2)).get
val z = MyClass("2").get

x: Int = 2
y: List[Int] = List(2)
z: String = "2"

Taking an example from https://twitter.github.io/scala_school/type-basics.html#parametricpoly we see that Scala has "rank-1" polymorphism, meaning that we can't define higher order functions that call polymorphic functions. For more see https://papl.cs.brown.edu/2014/para-poly.html
      

In [5]:
def makeListFrom[A](f: A => List[A]): List[A] = f(2)

cmd5.sc:1: type mismatch;
 found   : Int(2)
 required: A
def makeListFrom[A](f: A => List[A]): List[A] = f(2)
                                                  ^Compilation Failed

: 

## Exercise: Write the type signature of the following functions

In [7]:
def applyTwice(f: Int => Int, x: Int) = f(f(x))
// applyTwice: (Int => Int, Int) => Int
applyTwice _

cmd7.sc:3: not found: value applyTwice_
val res7_1 = applyTwice_
             ^Compilation Failed

: 

In [7]:
def applyTwiceCurried(f: Int => Int)(x : Int) = f(f(x))
// applyTwiceCurried(Int => Int) => Int => Int
applyTwiceCurried _

defined function applyTwiceCurried
res6_1: Int => Int => Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2444/0x0000000801ceb840@6a6b5e61

In [8]:
// List[A].map
// List[A] => (A => B) => List[B]
List(1) map (_: Int => String)

res7: Int => String => List[String] = ammonite.$sess.cmd7$Helper$$Lambda$2595/0x0000000801d57840@7d17f1d6

In [9]:
// List[A].filter
// List[A] => (A => Boolean) => List[A]
List(1) filter _

res8: Int => Boolean => List[Int] = ammonite.$sess.cmd8$Helper$$Lambda$2606/0x0000000801d5d840@53db11d2

In [12]:
// List[A].zip 
// List[A] => List[B] => List[(A, B)]
List(1) zip List(_: List[String])

res11: List[String] => List[(Int, List[String])] = ammonite.$sess.cmd11$Helper$$Lambda$2630/0x0000000801d6c040@210e9082

## Exercise: Write a function that corresponds to the following type signatures

In [14]:
// (Int => Int) => (Int => Int) => Int
def foo(f: Int => Int)(g: Int => Int): Int = f(g(2))
foo _

defined function foo
res13_1: Int => Int => Int => Int => Int = ammonite.$sess.cmd13$Helper$$Lambda$2642/0x0000000801d76040@32a8218b

In [16]:
// (Int, Int, Int) => (Int => Int)
def func(x: Int, y: Int, z: Int)

func _

cmd16.sc:1: not found: value f
def func(x: Int, y: Int, z: Int) = f(x: Int => Int)
                                   ^cmd16.sc:1: type mismatch;
 found   : Int
 required: Int => Int
def func(x: Int, y: Int, z: Int) = f(x: Int => Int)
                                     ^Compilation Failed

: 